In [1]:
# %pip install git+https://github.com/tensorflow/docs

import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import RiotWatcher, LolWatcher, ApiError

from utils import configuration

settings = configuration.settings
api_key = settings.api_key

In [2]:
lol_watcher = LolWatcher(api_key)
watcher = RiotWatcher(api_key)

my_region = 'na1'
my_summoner = 'insanity123'

me = lol_watcher.summoner.by_name(my_region, my_summoner)
my_ranked_stats = lol_watcher.league.by_summoner(my_region, me['id'])
print(my_ranked_stats)

[]


In [3]:
me

{'id': 'jg7S93hSw4KfuYyaks9xYzAFZ4w3bp_PSfT4LsHJi60A-SQ',
 'accountId': 'Ab7L0SV8v_nX7hVWwKV7IbEAbA0dj-C4e9PvMcPaU-7hf8o',
 'puuid': 'K8izf-1-ovMV0xlJ2qhoL7Amw98UG248C2xf8GGDVSqf73KklNrVoNN1PQe1R4Zb2JFsOFa_kSEzpg',
 'name': 'fury hawkx',
 'profileIconId': 745,
 'revisionDate': 1656557016000,
 'summonerLevel': 20}

In [4]:
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']
summoner_spells_version=versions['n']['summoner']
items_version=versions['n']['item']

current_champ_list = lol_watcher.data_dragon.champions(champions_version)

my_matches = lol_watcher.match.matchlist_by_puuid(my_region, me['puuid'], count=100)

In [5]:
len(my_matches)

9

In [6]:
last_match01 = lol_watcher.match.by_id(my_region,      my_matches[0])

In [7]:
# First we get the latest version of the game from data dragon
versions = lol_watcher.data_dragon.versions_for_region(my_region)
champions_version = versions['n']['champion']

# Lets get some champions
static_champ_list = lol_watcher.data_dragon.champions(champions_version)
static_summoners_list = lol_watcher.data_dragon.summoner_spells(champions_version)
# print(current_champ_list)

In [8]:
last_match01 #['info']['participants']

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_3747820277',
  'participants': ['ELOe8U6_n-Js9RfHLtDs4rDJuP_O4f3XrckXPnD0nnf0rdf1FMrAzgCAXAKSk70y4wyjzhJ4mCUu3A',
   'juouUq8MyAgVYaTnxtTCG2Xn8ir9vmXfgnKUgfnT-LRLZ_-NYlAlrChHSSl-ix59Mq-J9qrHDdZHuA',
   'tdZA5-QO4JHdRM_A8ooCbWABZf9sOwp_L1wliAODGb9GcPamlT-_KX9WMe-nyUROelxMTdBMEF9sIg',
   'K8izf-1-ovMV0xlJ2qhoL7Amw98UG248C2xf8GGDVSqf73KklNrVoNN1PQe1R4Zb2JFsOFa_kSEzpg',
   'whAWIYse07YFwvgeDIcSTnHpOl50UbaWRPdzLYw4KkDXKB299tOWDwrQUz5RPmhgomQHrM1fWRvasQ']},
 'info': {'gameCreation': 1610896033000,
  'gameDuration': 1289211,
  'gameId': 3747820277,
  'gameMode': 'CLASSIC',
  'gameName': 'teambuilder-match-3747820277',
  'gameStartTimestamp': 1610896146842,
  'gameType': 'MATCHED_GAME',
  'gameVersion': '11.1.352.5559',
  'mapId': 11,
  'participants': [{'assists': 10,
    'baronKills': 0,
    'bountyLevel': 7,
    'champExperience': 8797,
    'champLevel': 12,
    'championId': 37225015,
    'championName': 'DrMundo',
    'championTransform'

In [34]:
# static_summoners_list

In [35]:
champ_dict = {}
for key in static_champ_list['data']:
    row = static_champ_list['data'][key]
    champ_dict[row['key']] = row['id']

In [57]:
champ_dict

{'266': 'Aatrox',
 '103': 'Ahri',
 '84': 'Akali',
 '166': 'Akshan',
 '12': 'Alistar',
 '32': 'Amumu',
 '34': 'Anivia',
 '1': 'Annie',
 '523': 'Aphelios',
 '22': 'Ashe',
 '136': 'AurelionSol',
 '268': 'Azir',
 '432': 'Bard',
 '200': 'Belveth',
 '53': 'Blitzcrank',
 '63': 'Brand',
 '201': 'Braum',
 '51': 'Caitlyn',
 '164': 'Camille',
 '69': 'Cassiopeia',
 '31': 'Chogath',
 '42': 'Corki',
 '122': 'Darius',
 '131': 'Diana',
 '119': 'Draven',
 '36': 'DrMundo',
 '245': 'Ekko',
 '60': 'Elise',
 '28': 'Evelynn',
 '81': 'Ezreal',
 '9': 'Fiddlesticks',
 '114': 'Fiora',
 '105': 'Fizz',
 '3': 'Galio',
 '41': 'Gangplank',
 '86': 'Garen',
 '150': 'Gnar',
 '79': 'Gragas',
 '104': 'Graves',
 '887': 'Gwen',
 '120': 'Hecarim',
 '74': 'Heimerdinger',
 '420': 'Illaoi',
 '39': 'Irelia',
 '427': 'Ivern',
 '40': 'Janna',
 '59': 'JarvanIV',
 '24': 'Jax',
 '126': 'Jayce',
 '202': 'Jhin',
 '222': 'Jinx',
 '145': 'Kaisa',
 '429': 'Kalista',
 '43': 'Karma',
 '30': 'Karthus',
 '38': 'Kassadin',
 '55': 'Katarina',


In [36]:
summoners_dict = {}
for key in static_summoners_list['data']:
    row = static_summoners_list['data'][key]
    summoners_dict[row['key']] = row['id']

In [37]:
summoners_dict

{'21': 'SummonerBarrier',
 '1': 'SummonerBoost',
 '14': 'SummonerDot',
 '3': 'SummonerExhaust',
 '4': 'SummonerFlash',
 '6': 'SummonerHaste',
 '7': 'SummonerHeal',
 '13': 'SummonerMana',
 '30': 'SummonerPoroRecall',
 '31': 'SummonerPoroThrow',
 '11': 'SummonerSmite',
 '39': 'SummonerSnowURFSnowball_Mark',
 '32': 'SummonerSnowball',
 '12': 'SummonerTeleport',
 '54': 'Summoner_UltBookPlaceholder',
 '55': 'Summoner_UltBookSmitePlaceholder'}

In [69]:
n_games = 10
Games = {}
Game_duration = np.zeros(n_games)
Damage = np.zeros(n_games)
Gold = []
match_detail = None


j = 0
cont = 0
while cont < n_games:
    try:
        last_match = my_matches[cont]
        match_detail = lol_watcher.match.by_id(my_region, last_match)

        participants = []
        for row in match_detail['info']['participants']:
            participants_row = {}
            participants_row['champion'] = row['championId']
            participants_row['win'] = row['kills']
            participants_row['assists'] = row['assists']
            participants.append(participants_row)
            
        Games[j] = pd.DataFrame(participants)

        Summoner_name = []
        for row in match_detail['info']['participants']:
            Summoner_name_row = {}
            Summoner_name_row = row['summonerName']
            Summoner_name.append(Summoner_name_row)

        i = 0
        for row in participants:
            row['championName'] = champ_dict[str(row['champion'])]
            row['Summoner_name'] = Summoner_name[i]
            row['Summoner Spell 1'] = summoners_dict[str(row['summoner1Id'])]
            row['Summoner Spell 2'] = summoners_dict[str(row['summoner2Id'])]
            i += 1

        Games[j] = pd.DataFrame(participants)
        for index, row in Games[j].iterrows():
            if row['Summoner_name'] == my_summoner:
                Damage[j] = row['totalDamageDealt']
                Gold[j] = row['goldEarned']

        time.sleep(1)
        j += 1
        cont += 1

    except:
        cont += 1


In [80]:
Games[0]

,champion,win,assists
0,84,2,7
1,76,3,8
2,236,8,3
3,81,6,4
4,555,8,5
5,79,12,8
6,421,15,8
7,142,6,10
8,202,2,14
9,147,0,16


In [39]:
# last_match = my_matches[0]
# match_detail = lol_watcher.match.by_id(my_region, last_match)

In [40]:
Games

{0:    champion  win  assists
 0        84    2        7
 1        76    3        8
 2       236    8        3
 3        81    6        4
 4       555    8        5
 5        79   12        8
 6       421   15        8
 7       142    6       10
 8       202    2       14
 9       147    0       16}